**Ch20-21_Naive Bayes分类**

Source: 
[数据分析实战 45 讲](https://time.geekbang.org/column/intro/100021701?tab=intro)

---

## 贝叶斯原理：推断 + 修正

贝叶斯为了解决一个叫“逆向概率”问题写了一篇文章，尝试解答在没有太多可靠证据的情况下，怎样做出更符合数学逻辑的推测。“逆向概率”是相对“正向概率”而言：正向概率的问题很容易理解，比如已知袋子里面有 N 个球，不是黑球就是白球，其中 M 个是黑球，能知道摸出黑球的概率是多少，但这种情况往往是上帝视角，即了解了事情的全貌再做判断。在现实生活中，我们很难知道事情的全貌。

贝叶斯则从实际场景出发，提了一个问题：如果我们事先不知道袋子里面黑球和白球的比例，而是通过我们摸出来的球的颜色，能判断出袋子里面黑白球的比例么？贝叶斯原理与其他统计学推断方法截然不同，它是建立在主观判断的基础上：在我们不了解所有客观事实的情况下，同样可以先**估计**一个值，然后根据实际结果不断进行**修正**。

可以把概率模型的训练过程理解为求**参数估计**的过程。如果一个硬币在 10 次抛落中正面均朝上。那么你会想，这个硬币是均匀的可能性是多少？这里硬币均匀就是个参数，*似然函数就是用来衡量这个模型的参数*。

条件概率和贝叶斯公式以后再有点忘了就快去回顾数学笔记本T_T！！

# Naive Bayes: 朴素贝叶斯，就朴素在假设分类特征是相互独立的

![Bayes theorem](https://freeimghost.net/images/2025/05/16/556ae2a160ce37ca48a456b7dc61e564.png)

* 分类中，处于不同的几个属性下的比例就可当作分母的**P(Ai)（一共i个属性）**

* 要分成的**类别各自出现的概率**就是先验概率**P(Cj)（第j个类别）**

分类的输出就是给定某事物有某几个属性时，**最可能在哪个类别里**，即算出后验概率P(Cj|Ai), Cj对应不同几个类别时，**比较Cj是哪个类别时使这个后验概率最大**。似然值就是训练用到的数据中当某数据有Ai几个属性，属于某类别的比例。P(Cj)是固定的，所以训练模型时就找似然值和先验概率乘积最大的类别，分类Cj即为此类。

很显然这个思路是与前两节课的决策树很不同的，那个是看分到那个节点以后决策的*确定性/‘果断程度’*最高，用*纯度*衡量；这个就是算概率啦，很好区分的说。

课后思考问题是，如果我的女朋友，在我的手机里发现了和别的女人的暧昧短信，于是她开始思考了3个概率问题：

- 在没有任何情况下，出轨/没出轨的概率：先验概率（**数据要分类的类别**）
  
- 如果出轨了，那么手机里有暧昧短信的概率：似然值（已有条件/*已知类别*，看是否有这一**属性**。另外一个例子是有一男一女两同学，按照经验我们会觉得男同学高更likely；即已知类别（已知结果），推理属性）

- 在手机里发现了暧昧短信，认为出轨的概率：**后验概率（有属性，推是否在此类别）**

专栏里有这样一句话：“如果数据是连续的，通过样本计算出均值和方差，也就是得到正态分布的密度函数。有了密度函数，就可以把值代入，算出某一点的密度函数的值。” 然而PDF在某点的函数值，实际上为概率在该点的变化率而不是该点的概率。 所以原话中的“密度函数的值”大概不够严谨...? 严格来说应称为“概率密度值”，而非概率，但其相对大小能反映不同类别下观测值的plausibility，因此可以用于比较，相当于赋予了分类的权重。

## sklearn中的Naive Bayes

三种朴素贝叶斯分类算法：

- 高斯朴素贝叶斯（**GaussianNB**）：特征变量是连续变量，符合高斯分布，比如说人的身高，物体的长度。

- 多项式朴素贝叶斯（**MultinomialNB**）：特征变量是离散变量，符合多项分布，在文档分类中特征变量体现在一个单词出现的次数，或者是单词的TF-IDF值（评估某个词语对于一个文件集的重要程度，分别代表term frequency与inverse document frequency（TF-IDF为这俩相乘），IDF认为一个单词出现在的文档数越少，就越能通过这个单词把该文档和其他文档区分开，于是越适合分类。

- 伯努利朴素贝叶斯（**BernoulliNB**）特征变量是布尔变量，符合 0/1 分布，在文档分类中特征变量体现在单词是否出现。

这次文本分类使用多项式朴素贝叶斯。注意`alpha`参数（2333这回不是透明度了）是平滑参数，需要平滑的原因是如果一个单词在训练样本中没有出现，这个单词的概率就会被计算为 0。但训练集样本只是整体的抽样情况，我们不能因为一个事件没有观察到，就认为整个事件的概率为0。

- 当 alpha=1 时，使用的是Laplace平滑，就是采用加1的方式来统计没有出现过的单词的概率。这样当训练样本很大的时候，加1得到的概率变化可以忽略不计，也同时避免了零概率的问题。

- 当 0<alpha<1 时，使用的是Lidstone平滑。此时alpha越小，迭代次数越多，精度越高。

```Python
from sklearn.naive_bayes import MultinomialNB 
clf = MultinomialNB(alpha=0.001).fit(train_features, train_labels)

test_tf = TfidfVectorizer(stop_words=stop_words, max_df=0.5, vocabulary=train_vocabulary)
test_features=test_tf.fit_transform(test_contents)

predicted_labels=clf.predict(test_features)
```
然后预测完还是使用`sklearn.metrics.accuracy_score()`传入groundtruth评估分类模型。

In [5]:
# 朴素贝叶斯分类器进行文本分类。完整代码在PyCharm里面！！
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer()  # 实例化
# TfidfVectorizer(stop_words=[不需要用于分类的词], token_pattern=token_pattern,过滤规则,用regex)
documents = [
    'this is the bayes document',
    'this is the second second document',
    'and the third one',
    'is this the document'
]
tfidf_matrix = tfidf_vec.fit_transform(documents)   # 用的是该对象的方法
a = tfidf_vec.get_feature_names_out() # 本来这里老报错，结果是sklearn版本更新后方法名变了orz
# 这个方法直接从TfidfVectorizer对象的**内部词汇表**中提取出所有的特征名称(即词汇表中的词),按照字母顺序排列，和特征在矩阵中的列索引一一对应。
# 不需要依赖 tfidf_matrix，因为它基于的是fit过程中已经建立好的内部词汇表（vocabulary_ 属性）。
print('不重复的词:',a)
print('每个单词的ID:\n', tfidf_vec.vocabulary_)
print('每个单词的tfidf值:\n', tfidf_matrix.toarray())

不重复的词: ['and' 'bayes' 'document' 'is' 'one' 'second' 'the' 'third' 'this']
每个单词的ID:
 {'this': 8, 'is': 3, 'the': 6, 'bayes': 1, 'document': 2, 'second': 5, 'and': 0, 'third': 7, 'one': 4}
每个单词的tfidf值:
 [[0.         0.63314609 0.40412895 0.40412895 0.         0.
  0.33040189 0.         0.40412895]
 [0.         0.         0.27230147 0.27230147 0.         0.85322574
  0.22262429 0.         0.27230147]
 [0.55280532 0.         0.         0.         0.55280532 0.
  0.28847675 0.55280532 0.        ]
 [0.         0.         0.52210862 0.52210862 0.         0.
  0.42685801 0.         0.52210862]]
